In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [3]:
# 데이터 불러오기
merchant_df = pd.read_csv('../data/raw/big_data_set1_f.csv', encoding='cp949')
merchant_df.columns = ['가맹점구분번호', '가맹점주소', '가맹점명', '브랜드구분코드', '가맹점지역', '업종', '상권', '개설일', '폐업일']

In [ ]:
import requests
import time

KAKAO_API_KEY = 

ADDRESS_LIST = merchant_df_prc['가맹점주소']

SLEEP_TIME_SECONDS = 0.1

def geocode_and_reverse_geocode(address_list, api_key):
    results = []

    for i, address in enumerate(address_list):
        print(f"[{i + 1}/{len(address_list)}] 주소 처리 중: {address}")

        lon, lat = get_coord_from_addr(address, api_key)

        h_dong = None
        
        if lon and lat:
            h_dong = get_hdong_from_coord(lon, lat, api_key)

        results.append({
            '주소': address,
            '경도(lon)': lon,
            '위도(lat)': lat,
            '행정동': h_dong
        })

        time.sleep(SLEEP_TIME_SECONDS)
        
    return results

def get_coord_from_addr(address, api_key):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"query": address}
    
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        
        if data and 'documents' in data and data['documents']:
            doc = data['documents'][0]
            return float(doc['x']), float(doc['y'])
        
        print(f"  -> 경도/위도 변환 실패: 검색 결과 없음")
        return None, None
        
    except Exception as e:
        print(f"  -> 경도/위도 변환 실패 (오류): {e}")
        return None, None

def get_hdong_from_coord(lon, lat, api_key):
    url = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"x": str(lon), "y": str(lat)}

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        
        if data and 'documents' in data and data['documents']:
            for doc in data['documents']:
                if doc.get('region_type') == 'H':
                    return doc.get('region_3depth_name')
        
        print(f"  -> 행정동 변환 실패: 행정동 정보 없음")
        return None
        
    except Exception as e:
        print(f"  -> 행정동 변환 실패 (오류): {e}")
        return None

print("--- 주소 정보 처리 시작 ---")
final_data = geocode_and_reverse_geocode(ADDRESS_LIST, KAKAO_API_KEY)
print("--- 주소 정보 처리 완료 ---")

--- 주소 정보 처리 시작 ---
[1/4185] 주소 처리 중: 서울 성동구 마장동
[2/4185] 주소 처리 중: 서울 성동구 마장동
[3/4185] 주소 처리 중: 서울 성동구 마장동
[4/4185] 주소 처리 중: 서울 성동구 마장동
[5/4185] 주소 처리 중: 서울 성동구 마장동
[6/4185] 주소 처리 중: 서울 성동구 마장동
[7/4185] 주소 처리 중: 서울 성동구 마장동
[8/4185] 주소 처리 중: 서울 성동구 마장동
[9/4185] 주소 처리 중: 서울 성동구 사근동
[10/4185] 주소 처리 중: 서울 성동구 송정동
[11/4185] 주소 처리 중: 서울 성동구 용답동
[12/4185] 주소 처리 중: 서울 성동구 용답동
[13/4185] 주소 처리 중: 서울 성동구 행당동
[14/4185] 주소 처리 중: 서울 성동구 행당동
[15/4185] 주소 처리 중: 서울 성동구 홍익동
[16/4185] 주소 처리 중: 서울 성동구 홍익동
[17/4185] 주소 처리 중: 서울 성동구 행당1동
[18/4185] 주소 처리 중: 서울  성동구  도선동
[19/4185] 주소 처리 중: 서울  성동구  마장동
[20/4185] 주소 처리 중: 서울  성동구  마장동
[21/4185] 주소 처리 중: 서울  성동구  마장동
[22/4185] 주소 처리 중: 서울  성동구  마장동
[23/4185] 주소 처리 중: 서울  성동구  마장동
[24/4185] 주소 처리 중: 서울  성동구  마장동
[25/4185] 주소 처리 중: 서울  성동구  마장동
[26/4185] 주소 처리 중: 서울  성동구  마장동
[27/4185] 주소 처리 중: 서울  성동구  마장동
[28/4185] 주소 처리 중: 서울  성동구  마장동
[29/4185] 주소 처리 중: 서울  성동구  마장동
[30/4185] 주소 처리 중: 서울  성동구  마장동
[31/4185] 주소 처리 중: 서울  성동구  마장동
[32/4185] 주소 처리 중: 서울  성동구  마장

In [110]:
final_df = pd.DataFrame(final_data)

In [111]:
mct_df = pd.concat([merchant_df, final_df], axis=1).drop('주소', axis=1)

In [112]:
dong = ['금호1가동', '옥수동', '용답동', '용답동', '왕십리도선동', '왕십리도선동', '용답동', '용답동', '왕십리도선동', '용답동', '용답동', '용답동', '용답동', '마장동', '왕십리2동']

In [113]:
temp_idx = mct_df.loc[mct_df['행정동'].isna()].index

In [115]:
mct_df.loc[mct_df['행정동'].isna(), '행정동'] = dong

In [123]:
mct_df.to_csv('./행정동.csv', index=False)